# Part I. ETL Pipeline for Pre-Processing the Files

## Pre-Processing The Data Files

#### Importing Python packages 

In [100]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating a list of filepaths to process original event CSV data files

In [101]:
# checking the current working directory
print(os.getcwd())

# getting the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# creating a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# joining the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/Users/rsharma/Mine/Udacity/DataEngineeringWithAWS/Projects/data_modelling_with_apache_cassandra


In [102]:
file_path_list

['/Users/rsharma/Mine/Udacity/DataEngineeringWithAWS/Projects/data_modelling_with_apache_cassandra/event_data/2018-11-15-events.csv',
 '/Users/rsharma/Mine/Udacity/DataEngineeringWithAWS/Projects/data_modelling_with_apache_cassandra/event_data/2018-11-22-events.csv',
 '/Users/rsharma/Mine/Udacity/DataEngineeringWithAWS/Projects/data_modelling_with_apache_cassandra/event_data/2018-11-09-events.csv',
 '/Users/rsharma/Mine/Udacity/DataEngineeringWithAWS/Projects/data_modelling_with_apache_cassandra/event_data/2018-11-18-events.csv',
 '/Users/rsharma/Mine/Udacity/DataEngineeringWithAWS/Projects/data_modelling_with_apache_cassandra/event_data/2018-11-04-events.csv',
 '/Users/rsharma/Mine/Udacity/DataEngineeringWithAWS/Projects/data_modelling_with_apache_cassandra/event_data/2018-11-01-events.csv',
 '/Users/rsharma/Mine/Udacity/DataEngineeringWithAWS/Projects/data_modelling_with_apache_cassandra/event_data/2018-11-27-events.csv',
 '/Users/rsharma/Mine/Udacity/DataEngineeringWithAWS/Projects/

#### Processing the files to create the CSV data file that will be used for Apache Casssandra tables

In [103]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [104]:
# checking the number of rows in the CSV file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Completing the Apache Cassandra portion of the project. 

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Writing the Apache Cassandra code

#### Creating a Cluster

In [105]:
# I am running cassandra from docker, so I'll use the connection to (0.0.0.0) on port 32770

from cassandra.cluster import Cluster
cluster = Cluster(['0.0.0.0'], 32770)

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [106]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity_data_mod_cassandra 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)
except Exception as e:
    print(e)

#### Set Keyspace

In [107]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity_data_mod_cassandra')
except Exception as e:
    print(e)

### Create tables to run the following queries. 

Note: With Apache Cassandra we model the database tables on the queries we want to run.

## Create queries to ask the following three questions of the data

1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    
3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [ ]:
## column positions
# 0 artist 
# 1 firstName of user
# 2 gender of user
# 3 item number in session
# 4 last name of user
# 5 length of the song
# 6 level (paid or free song)
# 7 location of the user
# 8 sessionId
# 9 song title
# 10 userId

#### Query 1:  The artist, song title, and song's length in the music app history that was heard during sessionId = 338 and itemInSession = 4

In [108]:
# the query only asks for the artist name, song title, and song length with respect to the session id and item number in session, 
# so the table that satisfies this query will only have the columns that are needed as outputs and the columns that will be used for filtering the data,
# i.e., artist, song_title, song_length, session_id, item_in_session
# the PRIMARY KEY will be a composite key containing the session_id and item_in_session because these two columns already satisfy uniqueness for each record

query = "CREATE TABLE IF NOT EXISTS artist_songs "
query = query + "(artist text, song_title text, song_length float, session_id int, item_in_session int, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [109]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_songs (artist, song_title, song_length, session_id, item_in_session)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

In [110]:
query = "SELECT artist, song_title, song_length FROM artist_songs WHERE session_id = 338 AND item_in_session = 4"
try:
    results = session.execute(query)
except Exception as e:
    print(e)

pd.DataFrame(results._current_rows)

,artist,song_title,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


#### Query 2: The name of artist, song (sorted by itemInSession), and user (first and last name) for userid = 10 and sessionid = 182

In [111]:
# the query only asks for the artist name, song title, user firstname, and user lastname with respect to the user id and session id, 
# so the table that satisfies this query will only have the columns that are needed as outputs, the columns that will be used for filtering the data, 
# and the column needed for sorting the song titles
# i.e., artist, song_title, user_first_name, user_last_name, user_id, session_id, item_in_session
# the PRIMARY KEY will be a composite key containing the user_id, session_id, and item_in_session because the columns user_id and session_id already satisfy 
# uniqueness for each record and the column item_in_session will be used as clustering key for sorting the song titles

query = "CREATE TABLE IF NOT EXISTS artist_songs_users "
query = query + "(artist text, song_title text, session_id int, item_in_session int, user_id int, user_first_name text, user_last_name text, \
                    PRIMARY KEY (user_id, session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [112]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_songs_users (artist, song_title, session_id, item_in_session, user_id, user_first_name, user_last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], int(line[8]), int(line[3]), int(line[10]), line[1], line[4]))

In [113]:
query = "SELECT artist, song_title, user_first_name, user_last_name FROM artist_songs_users WHERE user_id = 10 AND session_id = 182"
try:
    results = session.execute(query)
except Exception as e:
    print(e)
    
pd.DataFrame(results._current_rows)

,artist,song_title,user_first_name,user_last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


#### Query 3: The user name (first and last) in the music app history who listened to the song 'All Hands Against His Own'

In [114]:
# the query only asks for the user firstname, and user lastname with respect to the song title, 
# so the table that satisfies this query will only have the columns that are needed as outputs, the column that will be used for filtering the data, 
# and the column needed for mainting the uniqueness of records
# i.e. user_first_name, user_last_name, song_title, user_id
# the PRIMARY KEY will be a composite key containing the song_title and the user_id because the column song_title alone will not be enough to maintain the
# uniqueness for each record

query = "CREATE TABLE IF NOT EXISTS user_songs "
query = query + "(user_id int, user_first_name text, user_last_name text, song_title text, PRIMARY KEY (song_title, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [115]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_songs (user_id, user_first_name, user_last_name, song_title)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (int(line[10]), line[1], line[4], line[9]))

In [116]:
query = "SELECT user_first_name, user_last_name FROM user_songs WHERE song_title = 'All Hands Against His Own'"
try:
    results = session.execute(query)
except Exception as e:
    print(e)
    
pd.DataFrame(results._current_rows)

,user_first_name,user_last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [4]:
## TO-DO: Drop the table before closing out the sessions

In [96]:
query = "DROP TABLE IF EXISTS artist_songs"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [97]:
query = "DROP TABLE IF EXISTS artist_songs_users"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [98]:
query = "DROP TABLE IF EXISTS user_songs"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [117]:
session.shutdown()
cluster.shutdown()